# Importação das bibliotecas

In [2]:
!pip install -q -U keras-tuner

     |████████████████████████████████| 133 kB 5.3 MB/s 


In [3]:
import os
import zipfile

import pandas as pd
import numpy as np

from matplotlib import pyplot as plt

from gensim.models import Word2Vec
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv1D,MaxPooling1D, GlobalAveragePooling1D, GlobalMaxPool1D, Embedding, Dense, Flatten, Dropout
from keras.callbacks import EarlyStopping
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
tf.random.set_seed(42)

# Carregando os datasets

In [7]:
df = pd.read_csv('/content/drive/MyDrive/ProjetoFinal_ProcessText/datasets/NoStem_StopwordKeep_dataset.csv')
df

,sent_rating,processed_text_KeepStopwords,data
0,negativo,Bom em questão de som ela é realmente boa no e...,2021-06-09 00:00:00
1,negativo,O produto não responde corretamente aos comand...,2020-11-26 00:00:00
2,negativo,Quem compra uma caixa de som pensa normalmente...,2021-01-05 00:00:00
3,negativo,A funcionalidade é muito boa Comprei as lâmpad...,2021-06-16 00:00:00
4,positivo,Excelente assistente som muito bom e limpo a A...,2020-12-10 00:00:00
...,...,...,...
3805,positivo,A mídia não pôde ser carregada Gostei de tudo ...,2021-07-06 00:00:00
3806,positivo,Bom dia comprei a echo dot preta e o cabo da t...,2021-05-04 00:00:00
3807,positivo,Amei a Alexa ela é incrível Porém fiquei chate...,2021-05-18 00:00:00
3808,positivo,Atendeu minhas expectativas Reconhece voz ente...,2021-01-09 00:00:00


# Processamento dos dados

## Treinando o Word2Vec com modelo pré-treinado



In [8]:
df['processed_text_KeepStopwords'] = df['processed_text_KeepStopwords'].apply(lambda x: x.lower())

In [9]:
tokenized_sents = [word_tokenize(i) for i in df['processed_text_KeepStopwords'].values]

In [1]:
# Download do Word2Vec CBOW de 100 vetores do repositório do NILC - USP
!wget --no-check-certificate \
http://143.107.183.175:22980/download.php?file=embeddings/word2vec/cbow_s100.zip \
-O /tmp/cbow_s100.zip

--2022-04-04 01:58:02--  http://143.107.183.175:22980/download.php?file=embeddings/word2vec/cbow_s100.zip
Connecting to 143.107.183.175:22980... connected.
HTTP request sent, awaiting response... 200 OK
Length: 326003567 (311M) [application/octet-stream]
Saving to: ‘/tmp/cbow_s100.zip’

/tmp/cbow_s100.zip  100%[===================>] 310.90M  2.53MB/s    in 75s     

2022-04-04 01:59:18 (4.15 MB/s) - ‘/tmp/cbow_s100.zip’ saved [326003567/326003567]



In [10]:
# Extração do arquivo .zip
with zipfile.ZipFile('/tmp/cbow_s100.zip', 'r') as zip_ref:
    zip_ref.extractall('/tmp/cbow')

In [11]:
model_noStem = Word2Vec(size=100, window=5, min_count=1, workers=-1)
model_noStem.build_vocab(tokenized_sents)
 
# assign the vectors to the vocabs that are in Google's pre-trained model and your sentences defined above.
# lockf needs to be set to 1.0 to allow continued training.
model_noStem.intersect_word2vec_format('/tmp/cbow/cbow_s100.txt', lockf=1.0)
 
# continue training with you own data
model_noStem.train(tokenized_sents, total_examples=3, epochs = 5)


(0, 0)

### Montagem do vetor com os embedings

In [12]:
X = []
max_len = 0

for i, row in df.iterrows():
  vetores = model_noStem[word_tokenize(row['processed_text_KeepStopwords'])]

  max_len = max(max_len, len(vetores))
  X.append(vetores)
print("max_len: {}".format(max_len))

# Padding
def padding(exemplos, dimension):
  results = np.zeros((len(exemplos), dimension, 100))
  for i, sequence in enumerate(exemplos):
    results[i, :len(sequence), :] = sequence
  return results
 
X = padding(X, max_len)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


max_len: 562


In [13]:
X.shape

(3810, 562, 100)

### Label encoding de y

In [14]:
y= df['sent_rating']
print(np.unique(y))
y = LabelEncoder().fit_transform(y)
np.unique(y)

['negativo' 'positivo']


array([0, 1])

### Divisão em treino e treino

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [16]:
print(X_train.shape, X_test.shape)

(3048, 562, 100) (762, 562, 100)


# Construção dos modelos

## Inicializando o modelo basico

In [17]:
model_base = Sequential()
model_base.add(Conv1D(128, 5, activation='relu', input_shape=(562,100)))
model_base.add(GlobalMaxPool1D())
model_base.add(Dropout(0.5))
model_base.add(Dense(10, activation='relu'))
model_base.add(Dense(1, activation='sigmoid'))
model_base.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 558, 128)          64128     
                                                                 
 global_max_pooling1d (Globa  (None, 128)              0         
 lMaxPooling1D)                                                  
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 10)                1290      
                                                                 
 dense_1 (Dense)             (None, 1)                 11        
                                                                 
Total params: 65,429
Trainable params: 65,429
Non-trainable params: 0
____________________________________________________

## Compilando o modelo

In [18]:
metrics = ['acc','Precision','Recall']
model_base.compile(
 optimizer = "adam",
 loss = "binary_crossentropy",
 metrics = metrics
)

## Treinando o modelo base

In [19]:
es = EarlyStopping(monitor='val_loss', mode='auto', patience = 2, verbose=1)

results = model_base.fit(
 X_train, y_train,
 epochs= 50,
 batch_size = 150,
 validation_split=0.2,
 callbacks = [es]
)

Epoch 1/50
17/17 [==============================] - 13s 113ms/step - loss: 0.6867 - acc: 0.6235 - precision: 0.8599 - recall: 0.6659 - val_loss: 0.4964 - val_acc: 0.8541 - val_precision: 0.8541 - val_recall: 1.0000
Epoch 2/50
17/17 [==============================] - 1s 82ms/step - loss: 0.4569 - acc: 0.8507 - precision: 0.8507 - recall: 1.0000 - val_loss: 0.4270 - val_acc: 0.8541 - val_precision: 0.8541 - val_recall: 1.0000
Epoch 3/50
17/17 [==============================] - 1s 75ms/step - loss: 0.4292 - acc: 0.8507 - precision: 0.8507 - recall: 1.0000 - val_loss: 0.4201 - val_acc: 0.8541 - val_precision: 0.8541 - val_recall: 1.0000
Epoch 4/50
17/17 [==============================] - 1s 79ms/step - loss: 0.4185 - acc: 0.8507 - precision: 0.8507 - recall: 1.0000 - val_loss: 0.4111 - val_acc: 0.8541 - val_precision: 0.8541 - val_recall: 1.0000
Epoch 5/50
17/17 [==============================] - 1s 75ms/step - loss: 0.4019 - acc: 0.8511 - precision: 0.8510 - recall: 1.0000 - val_loss: 0.4

## Teste do modelo base

In [20]:
test_result = model_base.evaluate(X_test,  y_test, verbose=2)

24/24 - 0s - loss: 0.3389 - acc: 0.8556 - precision: 0.8585 - recall: 0.9922 - 223ms/epoch - 9ms/step


In [21]:
precision = test_result[2]
recall = test_result[3]
f1_score = (2*recall*precision)/(recall+precision)
print('F1-Score:', f1_score)

F1-Score: 0.9205202345402334


## Otimização

### Inicializando função de otimização do modelo

In [22]:
def model_builder(hp):
  model_new = Sequential()

  hp_units1 = hp.Int('units', min_value=5, max_value=201, step=5)
  hp_units2 = hp.Int('units2', min_value=2, max_value=101, step=2)
  model_new.add(Conv1D(hp_units1, hp_units2, activation='relu', input_shape=(562,100)))

  model_new.add(GlobalMaxPool1D())

  hp_units3 = hp.Float('units3', min_value=0.1, max_value=0.9, step=0.1)
  model_new.add(Dropout(hp_units3))

  hp_units4 = hp.Int('units4', min_value=2, max_value=101, step=2)
  model_new.add(Dense(hp_units4, activation='relu'))

  model_new.add(Dense(1, activation='sigmoid'))

  hp_learning_rate = hp.Choice('learning_rate', values=[1e-1,1e-2, 1e-3, 1e-4,1e-5])

  model_new.compile(
      optimizer = keras.optimizers.Adam(learning_rate=hp_learning_rate),
      loss = "binary_crossentropy",
      metrics = metrics
      )

  return model_new

### Inicializando o otimizador

In [23]:
tuner = kt.Hyperband(model_builder,
                     objective=kt.Objective("val_acc", direction="max"),
                     max_epochs=20,
                     factor=3,
                     directory='/content/drive/MyDrive/ProjetoFinal_ProcessText/',
                     project_name='tune_CNN_2')

In [24]:
stop_early = EarlyStopping(monitor='val_loss', mode='auto', patience = 2)

### Busca pelo modelo otimizado

In [25]:
tuner.search(X_train, y_train, epochs=50, batch_size = 150, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

Trial 30 Complete [00h 00m 25s]
val_acc: 0.8655737638473511

Best val_acc So Far: 0.8655737638473511
Total elapsed time: 00h 06m 56s
INFO:tensorflow:Oracle triggered exit


### Treino do modelo otimizado

In [26]:
model_tuned = tuner.hypermodel.build(best_hps)
model_tuned.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_1 (Conv1D)           (None, 529, 50)           170050    
                                                                 
 global_max_pooling1d_1 (Glo  (None, 50)               0         
 balMaxPooling1D)                                                
                                                                 
 dropout_1 (Dropout)         (None, 50)                0         
                                                                 
 dense_2 (Dense)             (None, 62)                3162      
                                                                 
 dense_3 (Dense)             (None, 1)                 63        
                                                                 
Total params: 173,275
Trainable params: 173,275
Non-trainable params: 0
________________________________________________

In [27]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
history = model_tuned.fit(X_train, y_train, epochs=50, batch_size = 150, validation_split=0.2)

val_acc_per_epoch = history.history['val_acc']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
17/17 [==============================] - 3s 97ms/step - loss: 0.6014 - acc: 0.6981 - precision: 0.8570 - recall: 0.7743 - val_loss: 0.4639 - val_acc: 0.8541 - val_precision: 0.8541 - val_recall: 1.0000
Epoch 2/50
17/17 [==============================] - 1s 64ms/step - loss: 0.4507 - acc: 0.8507 - precision: 0.8507 - recall: 1.0000 - val_loss: 0.4374 - val_acc: 0.8541 - val_precision: 0.8541 - val_recall: 1.0000
Epoch 3/50
17/17 [==============================] - 1s 67ms/step - loss: 0.4138 - acc: 0.8507 - precision: 0.8507 - recall: 1.0000 - val_loss: 0.4165 - val_acc: 0.8541 - val_precision: 0.8541 - val_recall: 1.0000
Epoch 4/50
17/17 [==============================] - 1s 64ms/step - loss: 0.3805 - acc: 0.8507 - precision: 0.8507 - recall: 1.0000 - val_loss: 0.3902 - val_acc: 0.8541 - val_precision: 0.8541 - val_recall: 1.0000
Epoch 5/50
17/17 [==============================] - 1s 65ms/step - loss: 0.3322 - acc: 0.8532 - precision: 0.8528 - recall: 1.0000 - val_loss: 0.362

### Repetição do treino para uma quantidade ótima de épocas




In [28]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(X_train, y_train, epochs=best_epoch, batch_size = 150, validation_split=0.2)

Epoch 1/11
17/17 [==============================] - 3s 102ms/step - loss: 0.4746 - acc: 0.8503 - precision: 0.8506 - recall: 0.9995 - val_loss: 0.4248 - val_acc: 0.8541 - val_precision: 0.8541 - val_recall: 1.0000
Epoch 2/11
17/17 [==============================] - 1s 65ms/step - loss: 0.3805 - acc: 0.8507 - precision: 0.8507 - recall: 1.0000 - val_loss: 0.3838 - val_acc: 0.8541 - val_precision: 0.8541 - val_recall: 1.0000
Epoch 3/11
17/17 [==============================] - 1s 63ms/step - loss: 0.3060 - acc: 0.8687 - precision: 0.8663 - recall: 1.0000 - val_loss: 0.3484 - val_acc: 0.8541 - val_precision: 0.8553 - val_recall: 0.9981
Epoch 4/11
17/17 [==============================] - 1s 63ms/step - loss: 0.2184 - acc: 0.9151 - precision: 0.9114 - recall: 0.9971 - val_loss: 0.3425 - val_acc: 0.8656 - val_precision: 0.8652 - val_recall: 0.9981
Epoch 5/11
17/17 [==============================] - 1s 64ms/step - loss: 0.1430 - acc: 0.9573 - precision: 0.9527 - recall: 0.9995 - val_loss: 0.35

### Teste do modelo otimizado

In [29]:
eval_result = hypermodel.evaluate(X_test, y_test)

24/24 [==============================] - 0s 11ms/step - loss: 0.4938 - acc: 0.8635 - precision: 0.8665 - recall: 0.9907


In [30]:
precision = eval_result[2]
recall = eval_result[3]
f1_score = (2*recall*precision)/(recall+precision)
print('F1-Score:', f1_score)

F1-Score: 0.9244186073736061


## Salvando as predições junto ao dataframe original

In [31]:
y_pred = []
for predict in hypermodel.predict(X_test):
  if predict > 0.5:
    y_pred.append(1)
  else:
    y_pred.append(0)

In [32]:
_, predict_df, _, _ = train_test_split(df, y, test_size=0.2, random_state=42)
predict_df.insert(1, 'predict', y_pred)
predict_df.to_csv('/content/drive/MyDrive/ProjetoFinal_ProcessText/predict_datasets/CNN_predict.csv')